In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
#from langchain.schema import Document

from langchain_core.documents import Document

#from langchain_community.vectorstores import chroma
from langchain_chroma import Chroma
#utility imports
import numpy as np
from typing import List




In [6]:
sample_docs=[
"""
1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.
Connectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).
Key Contacts: 7022588703 / 6366576503.
The Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, 
designed to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, 
offering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.
 For 2026, the campus reported exceptional placement statistics, with highest packages reaching up to 80 LPA for B.Tech students and 
 average packages around 15 LPA for top branches. The Corporate and Industry Relations (CIR) cell is particularly active here, 
 facilitating recruitment with global giants like Amazon, SAP Labs, and Walmart Global Tech. While smaller in size compared to Coimbatore, 
 the campus provides modern laboratories, design tools, and clean, hygiene-focused hostels that are highly rated by students


""",

"""
2. Coimbatore Campus (Main Campus) 
Location: Nestled in Ettimadai, this is the largest campus, housing over 12,000 students and 1,500 faculty members.
Academic Range: Offers 120+ programs across UG, PG, and Doctoral levels, including Engineering, Business, and Social Work.
Research: Home to an Internal Quality Assurance Cell (IQAC) and numerous specialized research centers.
Key Contacts: 0422-2685509 / 0422-2685510.

The Coimbatore campus, established in 1994, serves as the administrative headquarters of the university and is a sprawling 400-acre "oasis" nestled 
in the Western Ghats at Ettimadai. As of 2026, it supports over 12,000 students and nearly 1,500 faculty members across 120+ undergraduate,
 postgraduate, and doctoral programs. The campus is renowned for its eco-friendly initiatives, including massive forestation that 
 transformed once-barren land into a lush environment home to exotic birds like peacocks. Infrastructure highlights include a built-up 
 area of 28 lakh square feet, eight hostels accommodating 5,000 students, and an Olympic-standard swimming pool (50m x 25m). 
 Academic life is centered around the Schools of Engineering and Business, alongside Departments of Social Work and Communication. 
 For the 2026 placement season, the campus maintains a strong record with top recruiters like Google, Microsoft, and Intel, reporting 
 a highest package of 60 LPA and an average of 9 LPA.
""",

"""
3. Amritapuri Campus 
Location: Located in Vallikavu, Kerala, spanning 80 acres along the coastline.
Structure: Features 6 prestigious schools and over 20 research centers.
Academic Focus: Known for blending science with spirituality, offering programs in Computing, Engineering, Biotechnology, and Ayurveda.
Key Dates (2026): Even semester classes for 2026 are scheduled to commence on January 19, 2026.
Key Contacts: 7034024264 / 0476-2805280.

Situated in the serene coastal village of Vallikavu, Kerala, the Amritapuri campus spans 80 acres and is a unique blend of
 modern science and spiritual values. It features over one million square feet of built-up area, housing six prestigious schools 
 and more than 20 specialized research centers. The campus is highly residential, with approximately 95 percent of students living in hostels, 
 many of which are located within the ashram premises. Academic programs cover diverse fields including Biotechnology, Ayurveda, Engineering,
   and Social Sciences. Amritapuri is a research powerhouse, hosting the Humanitarian Technology (HuT) Labs, which recently launched the IOT-based 
   "PRABHA" robot and is hosting the 2026 IEEE International Conference on Robotics and Mechatronics. Facilities for students include high-speed 
   computer labs with over 600 systems, extensive sports complexes for cricket and football, and strictly vegetarian dining that promotes a holistic
     lifestyle.

"""



]

In [7]:
import os

data_dir = "data1"
os.makedirs(data_dir, exist_ok=True)

for i, doc in enumerate(sample_docs):
    with open(f"{data_dir}/doc_{i}.txt", "w", encoding="utf-8") as f:
        f.write(doc)

print("Documents saved in:", data_dir)


Documents saved in: data1


In [8]:
#STEP1:  -----> DOCUMENT LOADING
from langchain_community.document_loaders import DirectoryLoader,TextLoader

# Load documents from directory
loader = DirectoryLoader(
    "data1", 
    glob="*.txt", 
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)
documents = loader.load()

print(f"Loaded {len(documents)} documents")
print(f"\nFirst document preview:")
print(documents[0].page_content[:200] + "...")


Loaded 3 documents

First document preview:

1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tec...


In [9]:
documents

[Document(metadata={'source': 'data1\\doc_0.txt'}, page_content="\n1. Bengaluru Campus (School of Engineering & Computing) \nLocation: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.\nAcademic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.\nConnectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).\nKey Contacts: 7022588703 / 6366576503.\nThe Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, \ndesigned to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, \noffering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.\n For 2026, the campus reported exceptional placement statistics, with highest packages reaching up to 80 LPA for B

In [10]:
#step2----->DOCUMENT SPLITTING
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Maximum size of each chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    length_function=len,
    separators=[" "]  # Hierarchy of separators
)
chunks=text_splitter.split_documents(documents)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")
print(f"\nChunk example:")
print(f"Content: {chunks[0].page_content[:150]}...")
print(f"Metadata: {chunks[0].metadata}")

Created 11 chunks from 3 documents

Chunk example:
Content: 1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academi...
Metadata: {'source': 'data1\\doc_0.txt'}


In [11]:
chunks

[Document(metadata={'source': 'data1\\doc_0.txt'}, page_content='1. Bengaluru Campus (School of Engineering & Computing) \nLocation: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.\nAcademic Focus: Offers B.Tech in 5 disciplines and M.Tech in 6 disciplines. Key departments include Engineering and Computing.\nConnectivity: Approximately 17 km from Majestic Bus Terminus/City Railway Station; accessible via BMTC routes 340 L (Majestic) and 341 B (KR Market).\nKey Contacts: 7022588703 / 6366576503.\nThe Bengaluru campus is a 50-acre'),
 Document(metadata={'source': 'data1\\doc_0.txt'}, page_content="/ 6366576503.\nThe Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, \ndesigned to cater to the needs of India's Silicon Valley. It specializes in high-demand technical fields, \noffering B.Tech programs in five disciplines and M.Tech in six, with a strong focus on Computer Science, Artificial Intelligence, and Cybersecurity.\

In [ ]:

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

persist_directory = "./chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory,
    collection_name="amrita_clg"
)

vectorstore.persist()

print("Vectors stored:", vectorstore._collection.count())


In [12]:
#just to load--without duplicating the vectors

persist_directory = "./chroma_db"

# Load existing vectorstore
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=OpenAIEmbeddings(),
    collection_name="amrita_clg"
)

print(f"Vector store has {vectorstore._collection.count()} vectors")


Vector store has 11 vectors


In [13]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(
    model_name="gpt-5-nano"
)


In [14]:
## Convert vector store to retriever
retriever=vectorstore.as_retriever(
    search_kwargs={"k":3} ## Retrieve top 3 relevant chunks
)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001FB7E3D4440>, search_kwargs={'k': 3})

In [15]:
##CREATING RAG CHAIN USING LCEL
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

In [16]:
# Create a custom prompt
from langchain_core.prompts import ChatPromptTemplate
custom_prompt = ChatPromptTemplate.from_template("""Use the following context to answer the question 
about the college and place. Donot hallucinate and do not guess anything.
be crystal clear to the asked question.
If you don't know the answer based on the context, say you don't know.
give the specific details and support.

Context:
{context}

Question: {query}

Answer:""")
custom_prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="Use the following context to answer the question \nabout the college and place. Donot hallucinate and do not guess anything.\nbe crystal clear to the asked question.\nIf you don't know the answer based on the context, say you don't know.\ngive the specific details and support.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"), additional_kwargs={})])

In [17]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001FB7E3D4440>, search_kwargs={'k': 3})

In [18]:
#output doc for the promts...(combining all the chunks from the retriever for the prompt.)
def combi(docs):
    return "\n".join(doc.page_content for doc in docs)


In [19]:
rag_chain_lcel=(
    {
        "context":retriever|combi,
        "query":RunnablePassthrough()
    }
    |custom_prompt
    |llm
    |StrOutputParser()
)
rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001FB7E3D4440>, search_kwargs={'k': 3})
           | RunnableLambda(combi),
  query: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template="Use the following context to answer the question \nabout the college and place. Donot hallucinate and do not guess anything.\nbe crystal clear to the asked question.\nIf you don't know the answer based on the context, say you don't know.\ngive the specific details and support.\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image

In [20]:
response=rag_chain_lcel.invoke("what is the location on amritapuri campus in 1000 words")
response

'The Amritapuri Campus is located in Vallikavu, a coastal village in the state of Kerala, India. The campus sits on an 80-acre site that runs along the coastline, offering a seaside setting as its physical backdrop.\n\nGeographic setting and scale\n- Location: Vallikavu, Kerala, India.\n- Physical footprint: 80 acres of land that coastline-side, giving the campus a broad, expansive feel in a rural, coastal environment.\n- Landscape and surroundings: The campus is described as being along the coast, which implies access to sea views and a maritime ambience that frames daily life on campus.\n\nCampus footprint and built environment\n- Built-up area: The campus comprises over one million square feet of built-up space. This substantial built environment supports a large, multi-institutional campus within a single location.\n- Institutional layout: The Amritapuri Campus houses six prestigious schools and more than 20 specialized research centers, all within the Vallikavu site. The extensive

In [ ]:
#to test query_rag_lcel()

In [21]:
from IPython.display import display, Markdown

display(Markdown(response))


The Amritapuri Campus is located in Vallikavu, a coastal village in the state of Kerala, India. The campus sits on an 80-acre site that runs along the coastline, offering a seaside setting as its physical backdrop.

Geographic setting and scale
- Location: Vallikavu, Kerala, India.
- Physical footprint: 80 acres of land that coastline-side, giving the campus a broad, expansive feel in a rural, coastal environment.
- Landscape and surroundings: The campus is described as being along the coast, which implies access to sea views and a maritime ambience that frames daily life on campus.

Campus footprint and built environment
- Built-up area: The campus comprises over one million square feet of built-up space. This substantial built environment supports a large, multi-institutional campus within a single location.
- Institutional layout: The Amritapuri Campus houses six prestigious schools and more than 20 specialized research centers, all within the Vallikavu site. The extensive built environment supports a diverse array of programs and scholarly activity under one campus umbrella.
- Residential character: The campus is highly residential, with approximately 95 percent of students living in hostels. Many of these hostels are located within the ashram premises, reinforcing a living-learning environment where study, community, and spiritual values are integrated on a daily basis.

Structure and academic focus in relation to location
- Schools and research: The campus’s physical layout is designed to support six schools and a network of more than 20 research centers. This implies a campus where departments and research facilities are clustered to create synergies in areas such as biotechnology, Ayurveda, engineering, computing, and social sciences.
- Academic programs tied to the environment: The Amritapuri Campus is noted for blending science with spirituality, reflecting its coastal, serene setting as part of its identity. Programs highlighted in the available context include Computing, Engineering, Biotechnology, Ayurveda, and related social sciences. While the exact program mix is described with some variation in the text, the location supports a broad, interdisciplinary blend that leverages the campus’s sizable facilities and research infrastructure.

Key facilities and research relevance tied to the location
- HuT Labs and research focus: The campus hosts Humanitarian Technology (HuT) Labs, underscoring its role as a research powerhouse situated in Vallikavu. The presence of these labs signals a location that emphasizes applied technology and impact-driven research.
- Notable projects and events: The HuT Labs have launched projects such as the IOT-based “PRABHA” robot and have hosted events like the 2026 IEEE International Conference on Robotics and Mechatronics. These activities reflect how the campus’s coastal location supports a practical, forward-looking research ecosystem.
- Campus facilities for students: The campus provides high-speed computer labs with more than 600 systems and extensive sports complexes for cricket and football. While these are general facilities, their scale is consistent with a large, coastal campus environment designed to accommodate a substantial student population living in hostels.

Local environment and campus culture tied to location
- Serene coastal setting: Descriptions repeatedly emphasize the “serene coastal village” of Vallikavu, suggesting a campus culture influenced by its proximity to the sea and its tranquil surroundings. This environment supports both rigorous academic work and contemplative or spiritual values, which the campus explicitly aims to blend with modern science.
- Religious and residential context: The reference to many hostels being located within ashram premises points to a living-learning environment where students reside close to campus facilities and the spiritual dimensions that the campus highlights as part of its identity.

Summary of location-related details
- The Amritapuri Campus is located in Vallikavu, Kerala, on an 80-acre coastal site.
- It features a vast built-up area (over one million square feet) and houses six schools and more than 20 research centers within the same campus footprint.
- The campus is highly residential (about 95% of students live in hostels), with many accommodations located within the ashram premises.
- Its coastal setting underpins a unique blend of modern science and spirituality, reflected in its programs and research initiatives.
- Notable facilities and activities tied to the location include the Humanitarian Technology (HuT) Labs, the PRABHA IOT robot project, and hosting the 2026 IEEE International Conference on Robotics and Mechatronics.
- Academic programs mentioned in connection with the location include Computing, Engineering, Biotechnology, Ayurveda, and Social Sciences, illustrating the campus’s diverse, interdisciplinary environment in a coastal, serene setting.

If you’re looking for the precise address, map coordinates, or nearby landmarks beyond Vallikavu, Kerala, those specifics aren’t provided in the context you shared. Based on the available information, the clear location detail is Vallikavu, Kerala, on an 80-acre coastal campus in India.

In [35]:
def query_rag(question):
    print(f"Question:{question}")
    print("-"*50)

    #1)pass the string directly (when using RunnablePassThrough)
    ans=rag_chain_lcel.invoke(question)
    #print(f"answer:{ans}")
    display(Markdown(ans))

    #2)printing the sources for the confirmation
    docs=retriever.invoke(question)
    for i,doc in enumerate(docs):
      print(f"\n source{i+1} \n")
      print(doc.page_content[:200])
    
    

In [37]:
res=query_rag(
  "Amrita Vishwa Vidyapeetham Bengaluru campus Sarjapur Road Kasavanahalli location in 500 words"
)


Question:Amrita Vishwa Vidyapeetham Bengaluru campus Sarjapur Road Kasavanahalli location in 500 words
--------------------------------------------------


Amrita Vishwa Vidyapeetham’s Bengaluru campus, specifically the School of Engineering & Computing, is located off Sarjapur Road in Kasavanahalli/Junnasandra, Bengaluru. The campus sits on a 50-acre site and is described as a dynamic tech hub designed to cater to India’s Silicon Valley. It emphasizes high-demand technical fields and aims to be a center for engineering and computing education and research.

Academic programs are a key feature of this campus. It offers B.Tech programs in five disciplines and M.Tech programs in six disciplines. The academic focus centers on Engineering and Computing, with notable emphasis on fields such as Computer Science, Artificial Intelligence, and Cybersecurity. This makes it a hub for students seeking advanced, technology-driven education aligned with current industry needs.

In terms of campus orientation and environment, the Bengaluru campus is presented as a modern, tech-forward institution positioned to foster innovation and applied learning in engineering and computing disciplines. The scale of the campus (50 acres) and its location near Bengaluru’s tech ecosystem are highlighted as part of its identity as a premium engineering and technology campus in the region.

For external accessibility and connectivity, the campus is about 17 kilometers from Majestic Bus Terminus/City Railway Station, making it reachable via public transport. Specifically, BMTC routes 340 L (Majestic) and 341 B (KR Market) provide viable transit options to reach the campus, which helps students and visitors access the campus from central Bengaluru and surrounding areas.

Placement outcomes for the Bengaluru campus have been noted for 2026, with highest packages reaching up to 80 LPA for B.Tech students. This placement statistic underscores the campus’s emphasis on industry relevance and successful student outcomes in high-demand technical domains.

Key contacts for the Bengaluru campus are provided for prospective students, parents, and visitors: 7022588703 and 6366576503. These numbers can be used to obtain admission guidance, program details, campus information, and other relevant assistance.

In summary, Amrita Vishwa Vidyapeetham’s Bengaluru campus on Sarjapur Road in Kasavanahalli/Junnasandra represents a 50-acre engineering and computing-focused institution, offering B.Tech and M.Tech programs across multiple disciplines, with a strong emphasis on Computer Science, AI, and Cybersecurity. It positions itself as a modern tech hub near Bengaluru’s Silicon Valley, with accessible public transport options and notable 2026 placement outcomes, supported by direct contact channels for further information.


 source1 

1. Bengaluru Campus (School of Engineering & Computing) 
Location: Situated off Sarjapur Road in Kasavanahalli/Junnasandra, spanning 50 acres.
Academic Focus: Offers B.Tech in 5 disciplines and M.Tech

 source2 

/ 6366576503.
The Bengaluru campus is a 50-acre dynamic tech hub located off Sarjapur Road in Kasavanahalli, 
designed to cater to the needs of India's Silicon Valley. It specializes in high-demand te

 source3 

3. Amritapuri Campus 
Location: Located in Vallikavu, Kerala, spanning 80 acres along the coastline.
Structure: Features 6 prestigious schools and over 20 research centers.
Academic Focus: Known for b
